In [1]:
# Import some libraries
import os
import numpy as np
import mne
import glob
import os
%matplotlib qt 

data_directory = 'C:/Users/mvmigem/Documents/data/project_1/localiser_dat/'
dir_list = glob.glob(data_directory+'*')
destination_directory = 'C:/Users/mvmigem/Documents/data/project_1/preprocessed/localiser/'


In [26]:

data_directory = 'C:/Users/mvmigem/Documents/data/project_1/localiser_dat/'
dir_list = glob.glob(data_directory+'*')
epochs = []
evokeds = []

i = 20


sub = int(dir_list[i].split('loc_')[1].split('.bdf')[0])
raw = mne.io.read_raw_bdf(dir_list[i], preload = True)
# Rename and adress channels
fix_chans = {'EXG1':'eye_above','EXG2':'eye_below',
            'EXG3':'eye_left','EXG4':'eye_right',
            'EXG5':'M1','EXG6':'M2'}
raw.rename_channels(fix_chans)
# we still have two exg channels which weren't actually recorded though (EXG7
# and EXG8) these are empty, so we'll drop them
raw.drop_channels(['EXG7', 'EXG8'])
print(raw.info['ch_names'])
# we'll also reset the channel types, so MNE knows what is 'brain' data
raw.set_channel_types({'M1':'eeg', 'M2':'eeg',
                    'eye_above':'eog', 'eye_below':'eog',
                    'eye_left':'eog', 'eye_right': 'eog'})
print(raw.info)
# Rereference to mastoids
raw.set_eeg_reference(ref_channels = ['M1','M2'])
# then drop them
raw.drop_channels(['M1','M2'])
# Select montage
montage = mne.channels.make_standard_montage('biosemi64')
# There is a mismatch between the names of the recording and the names of the montage
# This dict is to rename the channel names to fit the montage
mon_chnames = montage.ch_names
raw_chnames = raw.info['ch_names']
rename_channels = dict(zip(raw_chnames[:64],mon_chnames))
raw.rename_channels(rename_channels)
# Set montage
raw.set_montage(montage)
# Downsampling variables (logic -> https://mne.tools/stable/auto_tutorials/preprocessing/30_filtering_resampling.html#best-practices)
current_sfreq = raw.info['sfreq']
desired_sfreq = 256  # Hz
decim = np.round(current_sfreq / desired_sfreq).astype(int)
obtained_sfreq = current_sfreq / decim
lowpass_freq = obtained_sfreq / 3.
raw_filtered = raw.copy().notch_filter(freqs = 50, fir_design = 'firwin', verbose=None,n_jobs=-1)
raw_filtered = raw_filtered.copy().filter(l_freq=1, h_freq=lowpass_freq,n_jobs=-1)
# Plot to reject bad channels manually
raw_filtered.compute_psd().plot()
raw_filtered.plot(n_channels=64, block = True)
# Then intepolate bad channels
interp_filt_raw = raw_filtered.copy().interpolate_bads(reset_bads = True)
# Annotate events
events = mne.find_events(interp_filt_raw)
# Event dict
event_id = {        # This needs to be short in the online preprocess only 4 trigger markers (4 quads)
    'position1':80,'position2':81, 'position3':82,'position4':83, 
}

# Define your threshold in seconds
threshold_ms = 1000
sfreq = interp_filt_raw.info['sfreq']  # Sampling frequency of your data
threshold_samples = int(threshold_ms / 1000 * sfreq)

# Calculate differences between consecutive events
event_times = events[:, 0]  # Extract the sample index (first column) of each event
time_diffs = np.diff(event_times)

# Identify where time differences exceed the threshold
long_gaps = time_diffs > threshold_samples

# Find the periods where the distance exceeds the threshold
indices_exceeding_threshold = np.where(long_gaps)[0]

# Create a list to hold the annotations
annotations = []

for idx in indices_exceeding_threshold:
    start_sample = events[idx, 0] + (0.52*sfreq) # Start of the period + 500ms for preceding trial
    end_sample = events[idx + 1, 0] - 1 # End of the period (-1 to avoid removing trial trigger)
    start_time = (start_sample / sfreq)  # Convert sample index to time in seconds and add padding for epoch
    duration = (end_sample - start_sample) / sfreq  # Duration in seconds

    # Create an annotation
    annotation = mne.Annotations(onset=start_time,
                                 duration=duration,
                                 description=f'bad_calibration_gap')
    
    # Append annotation to the list
    annotations.append(annotation)

# Convert the list of annotations to a single mne.Annotations object
if annotations:
    combined_annotations = annotations[0]
    for annotation in annotations[1:]:
        combined_annotations += annotation
    
    # Add the annotations to the raw object
    interp_filt_raw.set_annotations(combined_annotations)
else:
    print("No gaps exceeding the threshold were found.")


# ICA
ica = mne.preprocessing.ICA(n_components = 0.99)
ica.fit(interp_filt_raw,decim=2, verbose='error', reject_by_annotation=True)
ica.plot_components()

interp_filt_raw.plot(events=events,n_channels=64,)


IndexError: list index out of range

In [25]:
# Save the rejected ica's
exclude_ica = [0,1,2,6,7,15,16,17]

# Exclude ica
ica.exclude=exclude_ica
ica.apply(interp_filt_raw)

# Epoch data around stim onset
epochs_stimlock = mne.Epochs(interp_filt_raw, events, event_id = event_id,
    tmin = -0.5, tmax = 0.5, proj = False, baseline = (None,0), decim=decim, #from previous cell
    detrend = None, verbose = True, reject_by_annotation= False, preload = True)


epochs.append(epochs_stimlock)
epochs_stimlock.save(f"C:/Users/mvmigem/Documents/data/project_1/preprocessed/localiser/sub{sub:02}_localiser-epo.fif", overwrite=True)


Applying ICA to Raw instance


    Transforming to ICA space (39 components)
    Zeroing out 8 ICA components
    Projecting back using 64 PCA components
Not setting metadata
240 matching events found
Setting baseline interval to [-0.5, 0.0] s
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 240 events and 513 original time points (prior to decimation) ...
1 bad epochs dropped
Overwriting existing file.
